## 추진배경

* 모든 정보가 하나의 ERP에 있음에도, 기능 별로 메뉴가 구분되어 있어 **비효율/불편함**
  * 주요기능 : L/C수신, L/C등록, 세금계산서 조회, 물품수령증 조회, 은행네고
* 모든 정보가 전자화되어있으나, 각 서류의 내용(주문번호 등)이 일치하는지 **눈으로 확인중**
  * 24자리 영문/숫자 혼합 등이 있으며, 틀리면 **물품대금을 받지 못하므로 확인이 매우 중요함**
* 각 서류, 행위마다 관련된 **법 조항이 있어 준수해야 함**
  * ex) XX서류가 발행되고 N일 내에 YY서류 제출
* **Dashboard**형태로 구성하여 한번에 정보조회를 하고, 주요정보 일치여부 확인을 자동화

>
**[용어설명]**<br>
**L/C** : **물품주문서**이자 **대금청구시 사용될 은행계좌**와 비슷한 역할<br>
      (Letter of Credit, 신용장)<br>
**물품수령증** : 고객이 물건을 정상수령하였다는 서류, 대금청구에 사용할 수 있다<br>
**은행네고** : 정상수령했다는 서류 등을 제출하여, 은행에 준비되어있는 물품대금을 받을 수 있다

## 효과
* 유저가 각 메뉴코드를 기억/저장할 필요없이 한번에 정보를 확인하여 소요시간 및 불편사항 제거
* 시스템 검증을 통한 Human error제거, 육안확인시간 감소 & 물품대금 정상입수

## github repository
[관련 github레포](https://github.com/KR9268/LocalLC_GUI)

## [세부내용] 구현내용 & 사용한 언어/패키지 등

(1) 정보가 파편화되어있어 하나의 Tool로써 확인하고 관리하기 위해서 `Streamlit` 기반으로 만듦
* 정보 저장 및 조회
  * 내부정보는 ERP에서 가져와서 db에 적재(SAP Scripting활용을 위한 `win32com` 사용)
  * 외부정보는 xml을 읽어서 Tag로 필요한 정보를 찾아 db에 적재(`Beautifulsoup`, `sqlite3` 사용)
    (외부정보라고는 하나, ERP에 저장되어있는 xml을 불러들여서 사용함)
  * 데이터 저장 및 최초 쿼리는 SQL문으로 가져오나, join등 필요한 사후처리는 `pandas`를 활용

(2) ERP에 직접 입력하는 등의 수작업을 자동으로 수행
* 자동화 기능
  * ERP 수주내역 등록(고정정보는 Master화, 변동정보는 Streamlit 텍스트박스 활용)
  * 준수사항(법령 등)의 자동체크
    * 특정 날짜 내에 완료해야한다던가, 일치해야하는 내용 등을 자동으로 검수
    * Streamlit의 table내 체크박스표기(True,False)를 활용하여 이상여부를 직관적으로 확인 가능
    * 사용자가 어떤 행동을 해야하는지 참고사항란을 통해 지시(연장요청, 수령증발행요청 등)
  * 보유내역 및 관리대상(작업이 완료되지 않은 건)의 Filter 기능 제공(드롭박스로 선택)
  * ERP의 ID, PW를 입력해두어 작업 자동화 수행
    * 개인PC에서만 사용하는 Tool이며, 표기는 ***과 같이 암호화 표기되어 관리

(3) 설계시 고려사항, 특이사항, 참고사항
* 추가/삭제/변경 등 변동될 수 있는 정보는 Hardcoding이 아닌 db형태로 저장
  * 오류 등 상황에 대비하여 실행시 기존 db를 복사해두는 로직 구현해두었으나, 자주 실행시 과생성되어 향후 수정 예정
* ERP관련 기능은 SAP메뉴(T-code)기준으로 함수화하여 관리
* 정보조회 관련 기능은 기능별 dataframe 생성/변환하는 방향으로 함수화하여 관리
* ERP제어(SAP Scripting) **주요기능을 구현한 'NERP_PI_LC'는 자체제작한 것**으로 정리하여 업로드 예정**(pip 미등록)**
* 수익자기준 주요 EDI코드(참고용 기록) : 내국신용장(LOCADV), 물품수령증(LOCRCT)
  * BeautifulSoup를 위해 정리해둔 딕셔너리(locrct_id, locadv_id)는 표준규격일 것으로 예상하여 재사용가능할 것으로 예상

(4) Local L/C에 대한 세부정보 참고가능한 사이트

[KTNET - 이용안내 - 상세업무절차](https://ulocallc.utradehub.or.kr/LocalCredit/index.jsp?sso=ok)
